# American Soundex Encoder - Assignment 2

Kevin Nolasco

Cabrini University

MCIS565 - Natural Language Processing

04/03/2022

# Introduction

The purpose of this notebook is to practice processing text data by applying the American Soundex algorithm to words. [According to the Soundex page on Wikipedia.org](https://en.wikipedia.org/wiki/Soundex), "Soundex is a phonetic algorithm for indexing names by sound, as pronounced in English". The algorithm is as follows:

    [b, f, p, v] → 1
    [c, g, j, k, q, s, x, z] → 2
    [d, t] → 3
    [l] → 4
    [m, n] → 5
    [r] → 6


- Save the first letter. Map all occurrences of a, e, i, o, u, y, h, w. to zero(0)
- Replace all consonants (include the first letter) with digits as in above.
- Replace all adjacent same digits with one digit, and then remove all the zero (0) digits
- If the saved letter's digit is the same as the resulting first digit, remove the digit (keep the letter).
- Append 3 zeros if result contains less than 3 digits. Remove all except first letter and 3 digits after it (This step same as [4.] in explanation above).

Below is the implementation of this algorithm by building a class. I wanted to challenge myself and make the class flexible enough to deal with a single input and a list of strings and I was able to accomplish that by taking advantage or list comprehensions and for loops.

In [152]:
class AmericanSoundex:
    """
    The steps taken from the wiki page are broken into methods for this class.
    Each class will have a definition taken straight from the wiki.
    """

    def __init__(self, input_word):
        """
        Save the first letter.
        Check type.
        """
        self.input_type = type(input_word)
        self.input_word = input_word
        self.raise_flag = False
        if self.input_type == str:
            self.words_raw = [self.input_word]
            if (len(self.input_word) < 2):
                print ('Please input a word that has 2 or more characters!')
                self.raise_flag = True
                return
        else:
            self.words_raw = self.input_word
            # check that all words are good length
            self.raise_flag = not all([len(word) >= 2 for word in self.words_raw])

            if self.raise_flag:
                bad_words = [word for word in self.words_raw if len(word) < 2]
                print('Please ensure all words have 2 or more characters!')
                print('The following inputs need to be corrected: {}'.format(', '.join(bad_words)))

        self.first_chars = [word[0] for word in self.words_raw]
        self.words = [word.lower() for word in self.words_raw]
        self.final_words = []
    
    def map_vowels(self):
        """
        Map all occurrences of a, e, i, o, u, y, h, w. to zero(0)
        """
        vowels = ['a','e','i','o','u','y','h','w']
        self.words_in_progress = [''.join([char if char not in vowels else '0' for char in word]) for word in self.words]

    def check_consonant(self, consonant):
        """
        used by .map_consonants()
        and .map_first_letter()
        """
        if consonant in ['b','f','p','v']:
            return '1'
        elif consonant in ['c','g','j','k','q','s','x','z']:
            return '2'
        elif consonant in ['d', 't']:
            return '3'
        elif consonant in ['l']:
            return '4'
        elif consonant in ['m','n']:
            return '5'
        elif consonant in ['r']:
            return '6'
        else:
            return consonant

    def map_consonants(self):
        """
        Replace all consonants (include the first letter) with digits as in [2.] above
        """
        self.words_in_progress = [''.join([self.check_consonant(char) for char in word]) for word in self.words_in_progress]

    def drop_adjacent_chars(self):
        """
        Replace all adjacent same digits with one digit
        word_in_progress will no longer contain first letter after this step
        """
        tmp_list = []
        for word in self.words_in_progress:
            ind = 1
            short_word = [char for char in word]
            while ind < len(short_word):
                if short_word[ind] == short_word[ind - 1]:
                    short_word.pop(ind)
                    ind -= 1
                ind += 1
            tmp_list.append(short_word[1:])

        self.words_in_progress = tmp_list
    
    def drop_zeros(self):
        """
        remove all the zero (0) digits
        """
        self.words_in_progress = [''.join([char for char in word if char != '0']) for word in self.words_in_progress]

    def map_first_letter(self):
        """
        used by .check_first_digit()
        """
        self.first_char_digits = [self.check_consonant(char) for char in self.first_chars]
    

    def check_first_digit(self):
        """
        If the saved letter's digit is the same as the resulting first digit, remove the digit (keep the letter).
        """
        self.map_first_letter()
        for ind, word in enumerate(self.words_in_progress):
            while word[0] == self.first_char_digits[ind]:
                self.words_in_progress[ind] = ''.join(char for i, char in enumerate(word) if i != 0)
    
    def padding_or_cropping(self):
        """
        Padding: Append 3 zeros if result contains less than 3 digits. 
        Cropping: Remove all except first letter and 3 digits after it
        """
        for ind, word in enumerate(self.words_in_progress):
            if len(word) < 3:
                self.final_words.append(self.first_chars[ind] + word.ljust(3, '0'))
            else:
                self.final_words.append(self.first_chars[ind]  + word[:3])

    def print_result(self):
        if len(self.words_raw) == 1:
            for ind, word in enumerate(self.words_raw):
                print('Original Word: {}'.format(word))
                print('American Soundex Encoded Word: {}'.format(self.final_words[ind]))
        else:
            for ind, word in enumerate(self.words_raw):
                print('=====================================\n')
                print('Original Word: {}'.format(word))
                print('American Soundex Encoded Word: {}'.format(self.final_words[ind]))
                print('\n=====================================\n')

    
    def encode_and_show_result(self):
        """
        only print results when the input is appropriate
        """
        if not self.raise_flag:
            self.map_vowels()
            self.map_consonants()
            self.drop_adjacent_chars()
            self.drop_zeros()
            self.check_first_digit()
            self.padding_or_cropping()
            self.print_result()
        else:
            print('Error: algorithm will not run if input word is less than 2 characters.')
            print('Please update your input and try again.')

### Examples

Let's take some examples to test the code above. We will build this object 3 times. The first will take a string as input, the second will take the a list containing a single string as input, and the third will take a list with 5 examples as input.

#### Single String

From the wikipage, we know that 'Honeyman' should be encoded as "H555". Let's see if the algorithm works as intended for this word.

In [153]:
my_encoder_with_string_input = AmericanSoundex('Honeyman')
my_encoder_with_string_input.encode_and_show_result()

Original Word: Honeyman
American Soundex Encoded Word: H555


Looks good!

#### List with Single String
I expect to see the same output as above.

In [154]:
my_encoder_with_single_string_list = AmericanSoundex(['Honeyman'])
my_encoder_with_single_string_list.encode_and_show_result()

Original Word: Honeyman
American Soundex Encoded Word: H555


Looks good again.

#### Multi String List Input

Now we will test the algorithm with multiple words at once. The new words are as follows:

- 'Ashcroft' which yields 'A226'
- 'Rupert' which yields 'R163'
- 'Robert' which yields 'R163' (same as Rupert)
- 'Pfister' which yields 'P236' not 'P123' (the first two letters have the same number and are coded once as 'P')

In [155]:
my_soundex_encoder_list = AmericanSoundex(['Honeyman', 'Ashcroft', 'Rupert', 'Robert', 'Pfister'])
my_soundex_encoder_list.encode_and_show_result()


Original Word: Honeyman
American Soundex Encoded Word: H555



Original Word: Ashcroft
American Soundex Encoded Word: A226



Original Word: Rupert
American Soundex Encoded Word: R163



Original Word: Robert
American Soundex Encoded Word: R163



Original Word: Pfister
American Soundex Encoded Word: P236




### Quick Error Checking

Let's do two examples to show that the algorithm will only accept good inputs.

The first will have an input that is too short, then we will correct it based on the message.

In [156]:
bad_input = AmericanSoundex('A')
bad_input.encode_and_show_result()

Please input a word that has 2 or more characters!
Error: algorithm will not run if input word is less than 2 characters.
Please update your input and try again.


In [157]:
corrected_input = AmericanSoundex('As')
corrected_input.encode_and_show_result()

Original Word: As
American Soundex Encoded Word: A200


Now let's pass a list as our input, the algorithm should tell us which words need correcting.

In [159]:
bad_inputs = AmericanSoundex(['This', 'is', 'a', 'test', 't'])
bad_inputs.encode_and_show_result()

Please ensure all words have 2 or more characters!
The following inputs need to be corrected: a, t
Error: algorithm will not run if input word is less than 2 characters.
Please update your input and try again.


In [160]:
# I will add an 'n' after 'a' even though it won't make grammatical sense
corrected_inputs = AmericanSoundex(['This', 'is', 'an', 'test'])
corrected_inputs.encode_and_show_result()


Original Word: This
American Soundex Encoded Word: T200



Original Word: is
American Soundex Encoded Word: i200



Original Word: an
American Soundex Encoded Word: a500



Original Word: test
American Soundex Encoded Word: t230




# Conclusion

As you can see above, I was able to implement the American Soundex algorithm with no errors and it can correctly handle many inputs at once. The algorithm is also able to handle bad inputs, and gives useful recommendations to the user so that the program can run correctly.

The method that took me the most effort was the .drop_adjacent_chars() method because it changes the length of the substring when a duplicate is found. The workaround for this is to decrease the index when a change is made, and then the method works perfectly.

Thank you for your time.